In [2]:
from dataclasses import replace
import os
import re
import glob
from black import diff
import numpy as np
from tqdm import tqdm
import pandas as pd
import shutil

In [1]:
# from itertools import islice

# def take(n, iterable):
#     "Return first n items of the iterable as a list"
#     return list(islice(iterable, n))

In [17]:
# Function to build a csv file with path name, image ID and identity
# Corrected lines of code to include appropriate directory, and split with extra condition to get ID
# Assertion error where identity is null - occurs because some images have 0013_left_eye etc.

def gen_celeba_csv(images='./dataset/celeba/images/1', identity='./dataset/celeba/identity_CelebA.txt'):
	imgs = [x for x in glob.glob(os.path.join(images, '*'))]
# 	print(imgs)
	csv = pd.DataFrame(imgs, columns=['path'])
	csv['id'] = csv['path'].str.split('/').str[-1]
	csv['id'] = csv['id'].str.split("\\").str[-1]
# 	display(csv)
	identity = {line.split()[0]:line.split()[1].strip() for line in open(identity).readlines()}
	attributes = [line.strip().split() for line in open('./dataset/celeba/list_attr_celeba.txt').readlines()[2:]]
	attri_cols = open('./dataset/celeba/list_attr_celeba.txt').readlines()[1].strip().split(' ')
# 	print(take(3, identity.items()))
	csv['identity'] = csv['id'].map(identity)
# 	display(csv[csv['identity'].isnull()])
# 	assert not csv['identity'].isnull().any()
	attributes = pd.DataFrame(attributes, columns=['id']+attri_cols)
	csv = csv.merge(attributes, on='id')
	csv[csv=='-1'] = 0
# 	display(csv)
	csv.to_csv('./dataset/celeba/data.csv', index=False)

In [40]:
def gen_celeba_csv(images='./dataset/celeba/images', identity='./dataset/celeba/identity_CelebA.txt'):
	imgs = [x for x in glob.glob(os.path.join(images, '*'))]
	csv = pd.DataFrame(imgs, columns=['path'])
	csv['id'] = csv['path'].str.split('/').str[-1]
	identity = {line.split()[0]:line.split()[1].strip() for line in open(identity).readlines()}
	attributes = [line.strip().split() for line in open('./dataset/celeba/list_attr_celeba.txt').readlines()[2:]]
	attri_cols = open('./dataset/celeba/list_attr_celeba.txt').readlines()[1].strip().split(' ')
	csv['identity'] = csv['id'].map(identity)
# 	assert not csv['identity'].isnull().any()
	attributes = pd.DataFrame(attributes, columns=['id']+attri_cols)
	csv = csv.merge(attributes, on='id')
	csv[csv=='-1'] = 0
	display(csv)
	csv.to_csv('./dataset/celeba/data.csv', index=False)

In [5]:
# tqdm used for monitoring progress
# csv file from gen_celeba_csv used for loading the file and encode it into a 128 dim vector

def gen_celeba_embeds(csv='./dataset/celeba/data.csv'):
	import face_recognition
	csv = pd.read_csv(csv)
	for path in tqdm(csv['path']):
		img = face_recognition.load_image_file(path)
		try:
			embed = face_recognition.face_encodings(img)[0]
# 			print(embed)            
		except:
			continue
		save = path.replace('images', 'embeds')+'.npy'
		folder = '/'.join(save.split('/')[:-1])
		if not os.path.exists(folder):
			os.makedirs(folder)

		np.save(save, embed)

In [19]:
def build_names():
	# creates a dictionary for each image (key) and the person appears on it (value)
	names_path = './dataset/celeba/identity_CelebA.txt'
	names = {}
	with open(os.path.join(os.getcwd(), names_path), 'r') as f:
		data = f.readlines()
		for i in data:
			pair = i.split()
			names.update({pair[0] : pair[1]})
	return names

# create a global variable that contains all people's names
all_names = build_names()
len(all_names)

202599

In [21]:
# Generates list of 'B' images' paths

def gen_img_batch(B = 3, names = all_names):
	import random

	# all image feat Nxd and the path for each image
	images ='./dataset/celeba/images'
	paths = [x for x in glob.glob(os.path.join(images, '*', '*')) if len(x.split('/')[-1].split('\\')[-1]) == 10]

	while True:
		# randomly select B images from paths
		l = random.sample(paths, B)

		duplicate_names = {}
		for i in l:
			img = i.split('/')[-1].split('\\')[-1]
			each_name = names.get(img)
			if each_name in duplicate_names:
				continue
			else:
				duplicate_names.update({each_name : 1})
		break
	return l

In [45]:
# shutil copy(src, dest) - syntax
# first condition only works if /pool has no contents - but had to create folders 1, 2 & 3
# second condition samples 30 images from pool into _pool

def gen_batches(num=1000):
	if len(os.listdir('./dataset/celeba/pool')) == 0:
		print('#')        
		res = []
		for _ in range(num):
			_res = []
			imgs = gen_img_batch(B=3)
			for x in imgs:
				_x = '/'.join(x.split('/')[:-2] + x.split('/')[-1:])
				shutil.copy(x, _x.replace('images', 'pool'))
				_res.append(_x.replace('images', 'pool'))
			res.append(_res)
		res = pd.DataFrame(res, columns=['img1', 'img2', 'img3'])
		res.to_csv('./dataset/celeba/pool.csv', index=False)
	else:
		csv = pd.read_csv('./dataset/celeba/pool.csv')
		csv.sample(30).to_csv('./dataset/celeba/_pool.csv', index=False)

In [48]:
# cleaned batch data seems to be crowdsourced with the question - which two faces are similar?
# valid adjs if adj occurs more than 'v' times in the fucntion get_valid_adjs (v=10)
# prints commonly occuring adjs related to facial components
# a1, a2 are the two similar images with same component compo, adjective adj1 while the same component of the third image has adj2

def gen_face_data_csv():
	def get_valid_adjs(csv):
		maps = csv['adj1'].value_counts().to_dict()
		maps.update(csv['adj2'].value_counts().to_dict())
		adjs = [k for k, v in maps.items() if v > 10]

		return adjs

	idx2compo = {i+1:x for i, x in enumerate(['forehead', 'eyebrow', 'eye', 'mouth', 'chin', 'cheek', 'nose', 'ear', 'temple', 'nostril', 'tooth', 'lip', 'tongue', 'skin'])}

	csv = pd.read_csv('./BatchData/cleaned_batch.csv')
	csv = csv[csv['Reject'].isnull()]
	#Removing images that have been rejected     
	csv = csv[['Input.img1', 'Input.img2', 'Input.img3', 'Answer.a1', 'Answer.a2', 'Answer.compo', 'Answer.d1', 'Answer.d2']].rename(
		columns={'Input.img1': 'img1', 'Input.img2': 'img2', 'Input.img3': 'img3',
		'Answer.a1': 'a1', 'Answer.a2': 'a2', 'Answer.compo': 'compo', 'Answer.d1': 'adj1', 'Answer.d2': 'adj2'}
	)
	csv['compo'] = csv['compo'].map(idx2compo)
	for adj in ['adj1', 'adj2']:
		csv[adj] = csv[adj].str.lower()
		csv[adj] = csv[adj].apply(lambda x: re.sub(r'[^\w\s]', '', x))
		csv[adj] = csv[adj].str.replace('white', 'light')
		csv[adj] = csv[adj].str.replace('lighter', 'light')
		csv[adj] = csv[adj].str.replace('darker', 'dark')
		csv[adj] = csv[adj].str.replace('black', 'dark')

	# with open('./dataset/celeba/valid_adjs.txt', 'w') as f:
	# 	for x in set(csv['adj1'].tolist()+csv['adj2'].tolist()):
	# 		f.write('{}\n'.format(x))

	valid_adjs = get_valid_adjs(csv)
	csv = csv[csv['adj1'].isin(valid_adjs)]
	csv = csv[csv['adj2'].isin(valid_adjs)]
	csv['a1'] -= 1
	csv['a2'] -= 1
	csv = csv[csv['a1']!=csv['a2']]

	csv = csv.groupby('compo').filter(lambda x: len(x)>10)
	for name, group in csv.groupby('compo'):
		print('------{}'.format(name))
		adjs = set(tuple(zip(group['adj1'], group['adj2'])))
		print('{} {} {} adjs for description'.format(name, len(group), len(adjs)))
		print(adjs)
	csv.to_csv('./dataset/celeba/crowd.csv', index=False)

In [41]:
if __name__ == '__main__':
	gen_celeba_csv()
# 	gen_celeba_embeds()
# 	build_names()
# 	l = gen_img_batch()
# 	gen_batches()
# 	gen_face_data_csv()

,path,id,identity,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
